In [1]:
import time

In [33]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as Soup

In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [35]:
def parse_page(tr_list):
    tmp = list()
    for tr in tr_list:
        tmp.append(list())
        for i, td in enumerate(tr.find_elements_by_tag_name('td')):
            if i == 0:
                tmp[-1].append(td.find_element_by_tag_name('a').get_attribute('href'))
            tmp[-1].append(td.text.strip())
    return pd.DataFrame(tmp, columns=column_names)

---

#### The only input

In [36]:
from_date = '01/01/1945'
to_date = '01/01/1949'

---

#### Set criteria

Some constant var

In [51]:
base_url = 'https://treaties.un.org'
column_names = ['href', 'title', 'reg_num', 'reg_date', 'type', 'con_date', 'vol']
tid = 'ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolderInnerPage_dgTreaty'
url = 'https://treaties.un.org/Pages/AdvanceSearch.aspx?tab=UNTS&clang=_en'

Open browser

In [38]:
browser = webdriver.Chrome()
browser.get(url)

In [39]:
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpSearchObj'))
select.select_by_visible_text('Treaty')
time.sleep(np.random.randint(1,2))
## Select Show Only Original Agreement
browser.find_element_by_id('ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolderInnerPage_rdbtreaty_2').send_keys(Keys.SPACE)
#time.sleep(np.random.randint(10, 15))

## Select filter by: Treaty type
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpAttribute'))
time.sleep(np.random.randint(1,2))
## the 20th one is `Treaty type`
select.select_by_index(20)
## Select "Open Multilateral"
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$chkboxValues$2').send_keys(Keys.SPACE)
#time.sleep(np.random.randint(10, 15))
time.sleep(np.random.randint(1,2))
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$Adv_srch2').send_keys(Keys.SPACE)

browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnAdd').send_keys(Keys.SPACE)

if '3' in from_date:
    from_date = from_date.replace('3', Keys.NUMPAD3)
if '3' in to_date:
    to_date = to_date.replace('3', Keys.NUMPAD3)

select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpAttribute'))
select.select_by_visible_text('Date of Registration')
from_field = browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$txtFrom')
from_field.click()
from_field.clear()
from_field.send_keys(from_date)
to_field = browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$txtTo')
to_field.click()
to_field.clear()
to_field.send_keys(to_date)

#time.sleep(np.random.randint(10, 15))
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnAdd').send_keys(Keys.SPACE)

##### Search

In [42]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnSubmit').send_keys(Keys.SPACE)

#### Iterate and save data

In [43]:
df = pd.DataFrame(columns=column_names)

In [44]:
i = 1
while True:
    tbody = browser.find_element_by_id(tid).find_element_by_tag_name('tbody')
    tr_list = tbody.find_elements_by_tag_name('tr')
    d = len(tr_list[1].find_elements_by_tag_name('td'))
    if d < 10:
        s = '0%d'%d
    else:
        s = str(s)
    id_ = 'ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty$ctl01$ctl%s'%(s)
    btn = WebDriverWait(browser,
                        np.random.randint(5,7)).until(EC.presence_of_element_located((By.NAME, id_)))
    df = df.append(parse_page(tr_list[3:-2]), ignore_index=True)
    print 'Page %d, now have %d rows'%(i, df.shape[0])
    if btn.get_attribute('disabled') is not None:
        break
    btn.click()
    i += 1
    time.sleep(np.random.randint(5, 12)+2)

Page 1, now have 10 rows
Page 2, now have 20 rows
Page 3, now have 25 rows


In [48]:
df.to_csv('untc_%s_%s.csv'%(from_date.replace('/', ''), to_date.replace('/', '')), 
          index=False, encoding='utf8')

Close

In [49]:
browser.close()